<img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logo.png" width="40%"  align="left">

In [ ]:
import pandas as pd
import os, copy
import numpy as np
import matplotlib.pyplot as plt
import salem
import seaborn as sns
from scipy import optimize
from scipy import stats
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colorbar import ColorbarBase
from salem.graphics import ExtendedNorm
%matplotlib inline

# Ref Table 

In [ ]:
# Get the RGI
import geopandas as gpd
import glob, os
import oggm
from oggm.utils import get_rgi_dir
frgi = '/home/mowglie/Documents/OGGM_Experiments/rgi60_allglaciers.csv'
rgi_dir = get_rgi_dir(version='6')
if not os.path.exists(frgi):
    # one time action only
    fs = list(sorted(glob.glob(rgi_dir + "/*/*_rgi60_*.shp")))[2:]
    out = []
    for f in fs:
        sh = gpd.read_file(f).set_index('RGIId')
        del sh['geometry']
        out.append(sh)
    mdf = pd.concat(out)
    mdf.to_csv(frgi)
mdf = pd.read_csv(frgi, index_col=0, converters={'Form': str, 'TermType': str, 'RGIFlag':str, 'BgnDate':str, 
                                                 'EndDate':str, 'O1Region': str, 'O2Region':str, 'Name':str})
mdf['RGI_REG'] = [rid.split('-')[1].split('.')[0] for rid in mdf.index]
# Read glacier attrs
gtkeys = {'0': 'Glacier',
          '1': 'Ice cap',
          '2': 'Perennial snowfield',
          '3': 'Seasonal snowfield',
          '9': 'Not assigned',
          }
ttkeys = {'0': 'Land-terminating',
          '1': 'Marine-terminating',
          '2': 'Lake-terminating',
          '3': 'Dry calving',
          '4': 'Regenerated',
          '5': 'Shelf-terminating',
          '9': 'Not assigned',
          }
mdf['GlacierType'] = [gtkeys[g] for g in mdf.Form]
mdf['TerminusType'] = [ttkeys[g] for g in mdf.TermType]
mdf['IsTidewater'] = [ttype in ['Marine-terminating', 'Lake-terminating'] for ttype in mdf.TerminusType]
mdf['RGIId'] = mdf.index.values

In [ ]:
mdfa = mdf.copy()

In [ ]:
mdf = mdf.loc[mdf.RGI_REG != '19']
print(len(mdf))

# Errors 

In [ ]:
dfe = pd.read_csv('/home/mowglie/disk/OGGM_Output/list_errors.csv', index_col=0)

# Read in glacier chars

In [ ]:
dd = '/home/mowglie/disk/OGGM_Output/run_output_summary'
rgi_regs = ['rgi_reg_{:02}'.format(p) for p in np.arange(1, 19)]
df = []
for r in rgi_regs:
    p = os.path.join(dd, r, 'glacier_characteristics.csv')
    _df = pd.read_csv(p, index_col=0, low_memory=False)
    _df['rgi_reg'] = r[-2:]
    df.append(_df)
df = pd.concat(df)

In [ ]:
df = df.loc[~df.index.isin(dfe.index)]
assert np.all(~df.inv_volume_km3.isnull())

In [ ]:
xlim, ylim = [1e-2, 1e4], [1e-5, 1e4]
xlim_exp, ylim_exp = [-2, 4], [-5, 4]

In [ ]:
ax = df.plot(kind='scatter', x='rgi_area_km2', y='inv_volume_km3')
ax.semilogx()
ax.semilogy()
ax.set_xlim(xlim)
ax.set_ylim(ylim);

### Median AAR 

In [ ]:
other_aar = df.tstar_aar / (df.tstar_aar + 1)

In [ ]:
other_aar.plot(kind='hist', bins=np.arange(101)/100);

In [ ]:
other_aar.median(), other_aar.mean() 

In [ ]:
xvas = np.array([0.01, 1, 10, 100, 1000, 10000])
vas = 0.034*(xvas**1.375)

## Fit VAS

In [ ]:
area = df.rgi_area_km2.values
ref_v = df.inv_volume_km3.values
def to_optimize(x):
    v = x[0]*(area**x[1])
    return np.sqrt(np.mean((v - ref_v)**2))
fit_sqrt = optimize.minimize(to_optimize, [1., 1.])
def to_optimize(x):
    v = x[0]*(area**x[1])
    return np.mean(np.abs(v - ref_v))
fit_abs = optimize.minimize(to_optimize, [1., 1.])
def to_optimize(x):
    v = x[0]*(area**x[1])
    return np.mean(np.abs(v - ref_v)/ref_v)
fit_rel = optimize.minimize(to_optimize, [1., 1.])

In [ ]:
# Fit in log space 
dfl = np.log(df[['inv_volume_km3', 'rgi_area_km2']])
slope, intercept, r_value, p_value, std_err = stats.linregress(dfl.rgi_area_km2.values, dfl.inv_volume_km3.values)

In [ ]:
print('VAS Obs', 1.375, 0.034)
print('linfit in log', slope, np.exp(intercept))
print('RMS fit', fit_sqrt['x'][1], fit_sqrt['x'][0])
print('ABS fit', fit_abs['x'][1], fit_abs['x'][0])
print('REL fit', fit_rel['x'][1], fit_rel['x'][0])

In [ ]:
fit_1 = fit_sqrt['x'][0] * (xvas ** fit_sqrt['x'][1])
fit_2 = fit_abs['x'][0] * (xvas ** fit_abs['x'][1])
fit_3 = fit_rel['x'][0] * (xvas ** fit_rel['x'][1])
fit_4 = np.exp(intercept) * (xvas ** slope)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.plot(xvas, vas, label='VAS')
ax.plot(xvas, fit_1, label='RMS')
ax.plot(xvas, fit_2, label='ABS')
ax.plot(xvas, fit_3, label='REL')
ax.plot(xvas, fit_4, label='LIN')
ax.semilogx()
ax.semilogy()
ax.set_xlim(xlim)
ax.set_ylim(ylim)
plt.legend();

## SLE 

In [ ]:
mdfa['inv_volume_km3'] = df.inv_volume_km3
ina = mdfa['inv_volume_km3'].isnull()
mdfa.loc[ina, 'inv_volume_km3'] = np.exp(intercept) * mdfa.loc[ina, 'Area'] ** slope
mdfa['vas_volume_km3'] = 0.034 * (mdfa.Area ** 1.375)

In [ ]:
per_reg = mdfa.groupby('RGI_REG')['Area', 'vas_volume_km3', 'inv_volume_km3'].sum()
regnames, _ = oggm.utils.parse_rgi_meta(version='6')
per_reg['Name'] = regnames.values
per_reg['Huss2012_vol'] = [20402., 1025, 34399, 9814, 19042, 4441, 9685, 256, 16839, 140, 
                           117, 61, 5026, 3241, 1312, 144, 6674, 70, 37517]
per_reg['Huss2012_slr'] = [50.7,2.5,85.4,24.4,47.3,11.0,24.0,0.6,41.8,0.3,0.3,0.2,12.5,8.0,3.3,0.4,16.6,0.21,93.1]
per_reg['Grinsted_slr'] = [44.6,2.6,61.7,15.2,47.0,8.7,13.3,0.8,33.8,0.5,0.3,0.2,23.7,9.5,4.1,0.3,11.7,0.3,75.1]
per_reg['Grinsted_vol_from_slr'] = per_reg['Grinsted_slr'] / 1000 / 900 * (362*1e6)

In [ ]:
per_reg.iloc[:-1][['Huss2012_vol', 'Grinsted_vol_from_slr', 'vas_volume_km3', 'inv_volume_km3']].sum()

## Plot 

In [ ]:
dfs = np.log(df[['inv_volume_km3', 'rgi_area_km2']])

In [ ]:
# Cmap norm
norm = ExtendedNorm([1, 10, 100, 200, 500, 1000], ncolors=256, extend='max')
cm = copy.deepcopy(plt.get_cmap('viridis'))
cm.set_under('white')

# Figure and plot
f, ax = plt.subplots(1, 1, figsize=(7, 5))

dfs.plot.hexbin(ax=ax, x="rgi_area_km2", y="inv_volume_km3", norm=norm, cmap=cm, 
                     colorbar=False, gridsize=70, linewidths=0.1);

# Fit
ax.plot(np.log(xvas), np.log(vas), '--', color='C1', label='VAS (Bahr et al.)', linewidth=1)
ax.plot(np.log(xvas), np.log(fit_4), '--', color='C3', label='VAS (default OGGM)', linewidth=1)

# Manipulate axes
ax.set_xlim(np.log(xlim))
ax.set_ylim(np.log(ylim))

xt = [10.**e for e in np.arange(xlim_exp[0], xlim_exp[1]+1)]
xl = ["10$^{"+"{:d}".format(int(x))+"}$" for x in np.arange(xlim_exp[0], xlim_exp[1]+1)]
ax.set_xticks(np.log(xt))
ax.set_xticklabels(xl)

yt = [10.**e for e in np.arange(ylim_exp[0], ylim_exp[1]+1)]
yl = ["10$^{"+"{:d}".format(int(x))+"}$" for x in np.arange(ylim_exp[0], ylim_exp[1]+1)]
ax.set_yticks(np.log(yt))
ax.set_yticklabels(yl)

# Legend
plt.legend();

ax.set_xlabel('Area (km$^{2}$)')
ax.set_ylabel('Volume (km$^{3}$)')

# Colorbar
cax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.5)
ColorbarBase(cax, extend='max', orientation='vertical', cmap=cm, norm=norm, label='N Glaciers');

## Deviations 

In [ ]:
df['oggm_vas'] = np.exp(intercept) * (df.rgi_area_km2 ** slope)
df['diff_to_oggm_vas'] = (df.inv_volume_km3 - df.oggm_vas) / df.oggm_vas

In [ ]:
df['diff_to_oggm_vas'].plot(kind='hist', bins=np.linspace(-3, 3, 100));

In [ ]:
df.plot(kind='hexbin', x='diff_to_oggm_vas', y='tstar_avg_prcpsol_max_elev', sharex=False, norm=LogNorm());

In [ ]:
df.plot(kind='hexbin', x='diff_to_oggm_vas', y='flowline_avg_slope', sharex=False, norm=LogNorm());

In [ ]:
import statsmodels.api as sm

In [ ]:
dfols = df[['flowline_avg_slope', 'tstar_avg_prcpsol_max_elev', 
            'tstar_avg_temp_min_elev', 'rgi_area_km2']].copy()
dfols = (dfols - dfols.mean()) / dfols.std()
dfols = sm.add_constant(dfols)

In [ ]:
mod = sm.OLS(df.diff_to_oggm_vas, dfols)
res = mod.fit()

In [ ]:
res.summary()

# Sensi 

In [ ]:
dd = '/home/mowglie/disk/OGGM_Output/run_output_summary'
rgi_regs = ['rgi_reg_{:02}'.format(p) for p in np.arange(1, 19)]

In [ ]:
patts = ['*istics_[0-9]*_nofs.csv', '*istics_[0-9]*_wfs.csv', '*rect_[0-9]*_nofs.csv', 
         '*parab_[0-9]*_nofs.csv', '*fsfac*.csv']
names = ['def', 'wfs', 'rec', 'parab', 'fsfac']
out = dict()
for n, pattern in zip(names, patts):

    exps = [f for f in sorted(glob.glob(dd+'/rgi_reg_01/' + pattern))]
    assert len(exps) == 25

    df_exp = []
    if n == 'def':
        a_fac = []
    for exp in exps:
        odf = []
        for r in rgi_regs:
            p = os.path.join(dd, r, os.path.basename(exp))
            _df = pd.read_csv(p, index_col=0, low_memory=False)
            _df = _df[['rgi_area_km2', 'inv_volume_km3', 'vas_volume_km3']].copy()
            _df['rgi_reg'] = r[-2:]
            odf.append(_df)
        odf = pd.concat(odf)
        odf = odf.loc[~odf.index.isin(dfe.index)]
        assert np.all(~odf.inv_volume_km3.isnull())
        df_exp.append(odf)
        if n == 'def':
            a_fac.append(float(exp.split('_')[-2])*0.1)
    out[n] = df_exp

In [ ]:
dftot = pd.DataFrame(index=a_fac)
dftot['Default'] = [dfi.inv_volume_km3.sum() * 1e-5 for dfi in out['def']]
dftot['Sliding'] = [dfi.inv_volume_km3.sum() * 1e-5 for dfi in out['wfs']]
# dftot['Sliding (FS fac)'] = [dfi.inv_volume_km3.sum() * 1e-5 for dfi in out['fsfac']]
dftot['Rectangular'] = [dfi.inv_volume_km3.sum() * 1e-5 for dfi in out['rec']]
dftot['Parabolic'] = [dfi.inv_volume_km3.sum() * 1e-5 for dfi in out['parab']]
dftot['VAS'] = [dfi.vas_volume_km3.sum() * 1e-5 for dfi in out['def']]

In [ ]:
dftot['HF2012'] = [per_reg.iloc[:-1].Huss2012_vol.sum() * 1e-5 for dfi in out['def']]
dftot['G2013'] = [per_reg.iloc[:-1].Grinsted_vol_from_slr.sum() * 1e-5 for dfi in out['def']]

In [ ]:
# sanity
dftot['VAS'].iloc[0] * 1e5 /  (362.5*1e6) * 1000

In [ ]:
dftot['Parabolic'] / dftot['Rectangular']
# dftot['Default'] / dftot['Rectangular']

In [ ]:
dftot.plot();
plt.ylabel('10$^5$ km$^3$');

In [ ]:
outf = dict()
for k in out.keys():
    df_fit = pd.DataFrame(index=a_fac)
    f1 = []
    f2 = []
    for dfi in out[k]:
        # Fit in log space 
        dfl = np.log(dfi[['inv_volume_km3', 'rgi_area_km2']])
        slope, intercept, _, _, _ = stats.linregress(dfl.rgi_area_km2.values, dfl.inv_volume_km3.values)
        f1.append(np.exp(intercept))
        f2.append(slope)
    df_fit['c'] = f1
    df_fit['gamma'] = f2
    outf[k] = df_fit

In [ ]:
for k in out.keys():
    outf[k].plot(secondary_y='c');
    plt.title(k);

# Final plot

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

# Figure and plot
pf = 0.85
_, (ax, ax2) = plt.subplots(2, 1, figsize=(5.5*pf, 8*pf))

# Cmap norm
norm = ExtendedNorm([1, 10, 100, 200, 500, 1000], ncolors=256, extend='max')
cm = copy.deepcopy(plt.get_cmap('viridis'))
cm.set_under('white')


dfs.plot.hexbin(ax=ax, x="rgi_area_km2", y="inv_volume_km3", norm=norm, cmap=cm, 
                     colorbar=False, gridsize=70, linewidths=0.1);

# Fit
ax.plot(np.log(xvas), np.log(vas), '--', color='C1', label='VAS (Bahr et al.)', linewidth=1.5)
ax.plot(np.log(xvas), np.log(fit_4), '--', color='C3', label='VAS (default OGGM)', linewidth=1.5)

# Manipulate axes
ax.set_xlim(np.log(xlim))
ax.set_ylim(np.log(ylim))

xt = [10.**e for e in np.arange(xlim_exp[0], xlim_exp[1]+1)]
xl = ["10$^{"+"{:d}".format(int(x))+"}$" for x in np.arange(xlim_exp[0], xlim_exp[1]+1)]
ax.set_xticks(np.log(xt))
ax.set_xticklabels(xl)

yt = [10.**e for e in np.arange(ylim_exp[0], ylim_exp[1]+1)]
yl = ["10$^{"+"{:d}".format(int(x))+"}$" for x in np.arange(ylim_exp[0], ylim_exp[1]+1)]
ax.set_yticks(np.log(yt))
ax.set_yticklabels(yl)

# Legend
ax.legend();

ax.set_xlabel('Area (km$^{2}$)')
ax.set_ylabel('Volume (km$^{3}$)')

# Colorbar
cax = inset_axes(ax, width="3%", height="30%", loc=4,
                 bbox_to_anchor=(-.15, 0.03, 1, 2), 
                 bbox_transform=ax.transAxes) 

ColorbarBase(cax, extend='max', orientation='vertical', cmap=cm, norm=norm);
cax.set_title('Glaciers', loc='left')

# Second plot
cs = plt.get_cmap('Purples')(np.linspace(0.4, 1, 4))
dftot[['Rectangular', 'Default', 'Sliding', 'Parabolic']].plot(ax=ax2, color=cs);
cs = plt.get_cmap('Reds')(np.linspace(0.4, 1, 3))
dftot[['VAS', 'HF2012', 'G2013']].plot(ax=ax2, color=cs, style=':', alpha=0.8);

ax2.set_ylim([0.5, 2.5])

ax2.legend(ncol=2)

ax2.set_xlabel('A factor')
ax2.set_ylabel('Total Volume (10$^5$ km$^{3}$)')

# Save
plt.tight_layout()
plt.savefig('/home/mowglie/disk/MyProjects/Papers/2017_OGGM/gmdd/figs/global_inv.pdf', dpi=300, bbox_inches='tight')

In [ ]:
len(dfs)